<a href="https://colab.research.google.com/github/amanas/google-colab/blob/master/bt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from datetime import datetime
import backtrader as bt





